In [ ]:
import seaborn as sns
import numpy as np
import polars as pl
import matplotlib.pyplot as plt

import pandas as pd
import sqlite3
import time

For this to work you need to read the plateform documentation.

-> Fill a /src/.env file with:
- CRAFT_AI_ENVIRONMENT_URL = "https://xxxxxxx.craft.ai"
- CRAFT_AI_SDK_TOKEN = "xxxxxxxxxxxxxxxxxxxxx"
- MY_REPO_URL = "git@github.com:xxxxxxxx.git"
- MY_SSH_KEY_FILEPATH = "xxxxxxxxx"

In [ ]:
from craft_ai_sdk import CraftAiSdk
import os
from dotenv import load_dotenv

load_dotenv()

CRAFT_AI_ENVIRONMENT_URL = os.getenv("CRAFT_AI_ENVIRONMENT_URL")
CRAFT_AI_SDK_TOKEN = os.getenv("CRAFT_AI_SDK_TOKEN")
MY_REPO_URL = os.getenv("MY_REPO_URL")

sdk = CraftAiSdk(sdk_token=CRAFT_AI_SDK_TOKEN, environment_url=CRAFT_AI_ENVIRONMENT_URL)


with open(os.getenv("MY_SSH_KEY_FILEPATH"), 'r') as file:
     private_key = file.read().rstrip()


n_step is the footer for step name and pipeline name. This notebook will first delete steps and pipeline with that footer then create a new one

The fct is the function name from datexim_fcts.py that the step will use.

In [ ]:
header = "datex"
footer = "1"
fct = "alib_epxl_anchors"

In [ ]:
step_name = header + "step-" + footer
pipe_name = header + "pipe-" + footer

---

In [ ]:
sdk.list_steps()

In [ ]:
if step_name in [x['name'] for x in sdk.list_steps()]:
    sdk.delete_step(step_name=step_name)

In [ ]:
try:
    sdk.delete_pipeline(pipeline_name=pipe_name)
except:
    print("Pipeline does not exist")

In [ ]:
#create Step
sdk.create_step(
	step_name=step_name,
	function_path="src/light/datexim_fcts.py",
	function_name=fct,#compute_shap_vals
	
	container_config={
		"dockerfile_path": "Dockerfile",
			"language": "python:3.8-slim",
			"repository_url": MY_REPO_URL,
			"repository_deploy_key": private_key,
			"included_folders":["src","requirements.txt"],
	}
)

In [ ]:
sdk.list_steps()

In [ ]:
i=0
while(sdk.list_steps()[-1]['status'] != "Ready"):#or specific step[x['status'] for x in sdk.list_steps() if x['name']=="datexstep-"+str(n_step)]
    time.sleep(1)
    i+=1
print(i)
sdk.list_steps()

In [ ]:
sdk.list_steps()

In [ ]:
sdk.create_pipeline(pipeline_name=pipe_name, step_name=step_name)

In [ ]:
exec_id = sdk.execute_pipeline(pipeline_name=pipe_name)
exec_id

In [ ]:
sdk.get_pipeline_execution(pipeline_name=pipe_name, execution_id=exec_id["execution_id"])

In [ ]:
i=0
while(sdk.list_pipeline_executions(pipeline_name=pipe_name)[0]['status'] == "Running"):
    time.sleep(1)
    i+=1
print(i)
sdk.list_pipeline_executions(pipeline_name=pipe_name)

In [ ]:
logs = sdk.get_pipeline_execution_logs(pipeline_name=pipe_name, execution_id=exec_id['execution_id'])
print('\n'.join(log["message"] for log in logs))

In [ ]:
logs = sdk.get_pipeline_execution_logs(pipeline_name=pipe_name, execution_id=sdk.list_pipeline_executions(pipe_name)[-1]["execution_id"])
print('\n'.join(log["message"] for log in logs))

In [ ]:
sdk.list_pipeline_executions(pipe_name)[-1]["execution_id"]

---